In [1]:
import pypsa
import numpy as np
import csv

In [2]:
# ALL DATA DEFINED IN THIS CELL

NUM_HOURS = 30*24

#marginal costs in $/MWh
marginal_costs = {"Wind" : 1,
                  "Coal" : 80}

#power plant capacities (nominal powers in MW) in each region (not necessarily realistic)
power_plant_p_nom = {"North" : {"Coal" : 5000},
                     "West" : {"Coal" : 5000},
                     "FarWest" : {"Coal" : 5000},
                     "NorthCentral" : {"Coal" : 5000},
                     "East" : {"Coal" : 5000},
                     "Coast" : {"Coal" : 5000},
                     "SouthCentral" : {"Coal" : 5000},
                     "South" : {"Coal" : 5000}
                    }

# region electrical loads in MW (not real)
loads = {"North" : 1000,
         "West" : 1000,
         "FarWest" : 000,
         "NorthCentral" : 1000,
         "East" : 1000,
         "SouthCentral" : 1000,
         "South" : 1000,
         "Coast" : 1000}

# wind capacity (from ERCOT data gathered by Yuji)
wind_capacity = {"North" : 2415,
                 "West" : 4603,
                 "FarWest" : 2790,
                 "NorthCentral" : 615,
                 "East" : 0,
                 "SouthCentral" : 0,
                 "South" : 2432,
                 "Coast" : 0}

# battery capacity
battery_capacity = {"North" : 500,
                 "West" : 500,
                 "FarWest" : 500,
                 "NorthCentral" : 500,
                 "East" : 500,
                 "SouthCentral" : 500,
                 "South" : 500,
                 "Coast" : 500}

# distances of transmission lines, in km
distances = {}
distances[('FarWest', 'South')] = 579
distances[('FarWest', 'West')] = 224
distances[('West', 'North')] = 195
distances[('North', 'NorthCentral')] = 198
distances[('East', 'NorthCentral')] = 146
distances[('East', 'Coast')] = 290
distances[('West', 'SouthCentral')] = 340
distances[('SouthCentral', 'Coast')] = 243
distances[('NorthCentral', 'SouthCentral')] = 241 #note: needs fixing
distances[('South', 'SouthCentral')] = 193
distances[('South', 'Coast')] = 391

capacity = 3000 # COMPLETELY MADE UP

In [3]:
# Reads wind data from file and places into wind_curve

wind_curve = np.zeros(NUM_HOURS)
wind_curves_file= open('wind_utilization_curves_2017.csv')
reader = csv.reader(wind_curves_file)
for row in range(NUM_HOURS):
    wind_curve[row] = float(reader.next()[0])
wind_curves_file.close()

In [4]:
network = pypsa.Network()

network.set_snapshots(range(NUM_HOURS))

regions = ['North', 'West', 'FarWest', 'NorthCentral', 'East', 'Coast', 'SouthCentral', 'South']

for region in regions:
    
    network.add("Bus", region)

    for tech in power_plant_p_nom[region]:
        network.add("Generator",
                    "{} {}".format(region, tech),
                    bus=region,
                    p_nom=power_plant_p_nom[region][tech],
                    marginal_cost=marginal_costs[tech])
    
    network.add("Generator",
                "{} Wind".format(region),
                bus=region,
                p_nom=wind_capacity[region],
                marginal_cost=marginal_costs["Wind"],
                p_max_pu=wind_curve)

    network.add("Load",
                "{} load".format(region),
                bus=region,
                p_set=loads[region])
    
    network.add("StorageUnit",
            "{} Battery".format(region),
            bus=region,
            p_nom=battery_capacity[region],
            cyclic_state_of_charge = True)
    
for (region1, region2), d in distances.items():
    network.add("Link",
                "{} - {} link".format(region1, region2),
                bus0=region1,
                bus1=region2,
                p_nom=capacity,
                p_min_pu=-1)

In [9]:
#def extra_functionality(network, snapshots):
#    network.generators.marginal_cost

network.lopf()

/Applications/anaconda/envs/pypsa_env/lib/python2.7/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)
INFO:pypsa.pf:Slack bus for sub-network 0 is North
INFO:pypsa.pf:Slack bus for sub-network 1 is West
INFO:pypsa.pf:Slack bus for sub-network 2 is FarWest
INFO:pypsa.pf:Slack bus for sub-network 3 is NorthCentral
INFO:pypsa.pf:Slack bus for sub-network 4 is East
INFO:pypsa.pf:Slack bus for sub-network 5 is Coast
INFO:pypsa.pf:Slack bus for sub-network 6 is SouthCentral
INFO:pypsa.pf:Slack bus for sub-network 7 is South
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using glpk
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 139119776.44
  Upper bound: 139119776.44
  Number of objectives: 1
  Number of constraints: 38881
  Number of variables: 42481
  Number of nonzeros: 89281
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 5.77954101562
# ----------------------------------------------------------
#   Solution Information
# -----------------

('ok', 'optimal')

In [17]:
print network.model.__repr__()

In [6]:
wind_cap_total = 0
wind_use_total = 0

for region in regions:
    regional_wind_cap_total = np.sum(wind_curve * wind_capacity[region])
    regional_wind_use_total = np.sum(network.generators_t['p'][region + ' Wind'])
    wind_cap_total += regional_wind_cap_total
    wind_use_total += regional_wind_use_total
    print region, regional_wind_cap_total, regional_wind_use_total, regional_wind_use_total/regional_wind_cap_total

print 'Curtailment: ', 1 - wind_use_total / wind_cap_total

North 664315.7849999999 549624.5750000001 0.8273543808687311
West 1266188.637 1266188.6369999999 0.9999999999999998
FarWest 767470.41 702743.9920000001 0.9156626533653591
NorthCentral 169173.58500000002 161489.94 0.954581295892027
East 0.0 0.0 nan
Coast 0.0 0.0 nan
SouthCentral 0.0 0.0 nan
South 668992.128 662740.496 0.9906551486357699
Curtailment:  0.054679078090772104


/Applications/anaconda/envs/pypsa_env/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


In [13]:
network.generators.marginal_cost

North Coal           80.0
North Wind            1.0
West Coal            80.0
West Wind             1.0
FarWest Coal         80.0
FarWest Wind          1.0
NorthCentral Coal    80.0
NorthCentral Wind     1.0
East Coal            80.0
East Wind             1.0
Coast Coal           80.0
Coast Wind            1.0
SouthCentral Coal    80.0
SouthCentral Wind     1.0
South Coal           80.0
South Wind            1.0
Name: marginal_cost, dtype: float64

In [8]:
network.loads_t.p

,North load,West load,FarWest load,NorthCentral load,East load,Coast load,SouthCentral load,South load
0,1000.0,1000.0,0.0,1000.0,1000.0,1000.0,1000.0,1000.0
1,1000.0,1000.0,0.0,1000.0,1000.0,1000.0,1000.0,1000.0
2,1000.0,1000.0,0.0,1000.0,1000.0,1000.0,1000.0,1000.0
3,1000.0,1000.0,0.0,1000.0,1000.0,1000.0,1000.0,1000.0
4,1000.0,1000.0,0.0,1000.0,1000.0,1000.0,1000.0,1000.0
5,1000.0,1000.0,0.0,1000.0,1000.0,1000.0,1000.0,1000.0
6,1000.0,1000.0,0.0,1000.0,1000.0,1000.0,1000.0,1000.0
7,1000.0,1000.0,0.0,1000.0,1000.0,1000.0,1000.0,1000.0
8,1000.0,1000.0,0.0,1000.0,1000.0,1000.0,1000.0,1000.0
9,1000.0,1000.0,0.0,1000.0,1000.0,1000.0,1000.0,1000.0


In [9]:
network.generators_t.p

,North Coal,North Wind,West Coal,West Wind,FarWest Coal,FarWest Wind,NorthCentral Coal,NorthCentral Wind,East Coal,East Wind,Coast Coal,Coast Wind,SouthCentral Coal,SouthCentral Wind,South Coal,South Wind
0,0.0,282.555,0.0,538.551,0.0,326.43,0.000,71.955,0.0,0.0,0.000,0.0,3715.456,0.0,0.0,284.544
1,0.0,280.140,0.0,533.948,0.0,323.64,1790.932,71.340,0.0,0.0,2717.888,0.0,1000.000,0.0,0.0,282.112
2,0.0,301.875,0.0,575.375,0.0,348.75,1697.125,76.875,0.0,0.0,2696.000,0.0,1000.000,0.0,0.0,304.000
3,0.0,355.005,0.0,676.641,0.0,410.13,1467.819,90.405,0.0,0.0,2642.496,0.0,1000.000,0.0,0.0,357.504
4,0.0,338.100,0.0,644.420,0.0,390.60,1540.780,86.100,0.0,0.0,2659.520,0.0,1000.000,0.0,0.0,340.480
5,0.0,359.835,0.0,685.847,0.0,415.71,1446.973,91.635,0.0,0.0,2637.632,0.0,1000.000,0.0,0.0,362.368
6,0.0,434.700,0.0,828.540,0.0,502.20,1123.860,110.700,0.0,0.0,2562.240,0.0,1000.000,0.0,0.0,437.760
7,0.0,511.980,0.0,975.836,0.0,591.48,790.324,130.380,0.0,0.0,2484.416,0.0,1000.000,0.0,0.0,515.584
8,0.0,673.785,0.0,1284.237,0.0,778.41,91.983,171.585,0.0,0.0,2321.472,0.0,1000.000,0.0,0.0,678.528
9,0.0,777.630,0.0,1482.166,0.0,898.38,0.000,198.030,0.0,0.0,0.000,0.0,2860.690,0.0,0.0,783.104


In [10]:
network.buses_t.marginal_price

,North,West,FarWest,NorthCentral,East,Coast,SouthCentral,South
0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0
1,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0
2,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0
3,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0
4,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0
5,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0
6,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0
7,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0
8,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0
9,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0


In [11]:
network.links_t.p0

,South - Coast link,East - Coast link,East - NorthCentral link,West - SouthCentral link,NorthCentral - SouthCentral link,North - NorthCentral link,FarWest - West link,West - North link,SouthCentral - Coast link,FarWest - South link,South - SouthCentral link
0,2000.000,2000.00,-3000.0,-3000.000,-3000.000,428.045,-2173.570,645.490,-3000.0,3000.0,284.544
1,-717.888,2000.00,-3000.0,-3000.000,-3000.000,-862.272,-2676.360,-142.412,-3000.0,3000.0,3000.000
2,-696.000,2000.00,-3000.0,-3000.000,-3000.000,-774.000,-2651.250,-75.875,-3000.0,3000.0,3000.000
3,-642.496,2000.00,-3000.0,-3000.000,-3000.000,-558.224,-2589.870,86.771,-3000.0,3000.0,3000.000
4,-659.520,2000.00,-3000.0,-3000.000,-3000.000,-626.880,-2609.400,35.020,-3000.0,3000.0,3000.000
5,-637.632,2000.00,-3000.0,-3000.000,-3000.000,-538.608,-2584.290,101.557,-3000.0,3000.0,3000.000
6,-562.240,2000.00,-3000.0,-3000.000,-3000.000,-234.560,-2497.800,330.740,-3000.0,3000.0,3000.000
7,-484.416,2000.00,-3000.0,-3000.000,-3000.000,79.296,-2408.520,567.316,-3000.0,3000.0,3000.000
8,-321.472,2000.00,-3000.0,-3000.000,-3000.000,736.432,-2221.590,1062.647,-3000.0,3000.0,3000.000
9,2000.000,2000.00,-3000.0,-3000.000,-2643.794,1158.176,-2101.620,1380.546,-3000.0,3000.0,783.104


In [12]:
network.storage_units_t.state_of_charge

,North Battery,West Battery,FarWest Battery,NorthCentral Battery,East Battery,Coast Battery,SouthCentral Battery,South Battery
0,0.0,0.000,0.000,0.000,0.00,0.00,0.000,0.0
1,0.0,0.000,0.000,0.000,0.00,0.00,0.000,0.0
2,0.0,0.000,0.000,0.000,0.00,0.00,0.000,0.0
3,0.0,0.000,0.000,0.000,0.00,0.00,0.000,0.0
4,0.0,0.000,0.000,0.000,0.00,0.00,0.000,0.0
5,0.0,0.000,0.000,0.000,0.00,0.00,0.000,0.0
6,0.0,0.000,0.000,0.000,0.00,0.00,0.000,0.0
7,0.0,0.000,0.000,0.000,0.00,0.00,0.000,0.0
8,0.0,0.000,0.000,0.000,0.00,0.00,0.000,0.0
9,0.0,0.000,0.000,0.000,0.00,0.00,0.000,0.0
